## Importación de librerías y carga de datos

In [1]:
import pandas as pd

# Cargar dataset base
df_base = pd.read_csv('../data/processed/base_index.csv')
df_base.head()


,country,indicator,value
0,Brazil,current_account,-2.204252
1,Brazil,external_debt,32.770340
2,Brazil,gdp_growth,1.726360
3,Brazil,inflation,5.824015
4,Brazil,reserves_months,11.221928


##Asignaciondedireccionderiesgo

In [2]:
# True = mayor valor implica mayor riesgo
risk_direction = {
    'inflation': True,
    'external_debt': True,
    'current_account': False,  # superávit = menos riesgo
    'gdp_growth': False,       # más crecimiento = menos riesgo
    'reserves_months': False   # más reservas = menos riesgo
}


##

In [3]:
# Convertir a formato wide
df_pivot = df_base.pivot(index='country', columns='indicator', values='value')
df_norm = pd.DataFrame(index=df_pivot.index)

for col in df_pivot.columns:
    col_min, col_max = df_pivot[col].min(), df_pivot[col].max()
    if risk_direction[col]:
        df_norm[col] = (df_pivot[col] - col_min) / (col_max - col_min)
    else:
        df_norm[col] = (col_max - df_pivot[col]) / (col_max - col_min)


In [4]:
weights = {
    col: 1 / len(df_norm.columns) for col in df_norm.columns
}
weights


{'current_account': 0.2,
 'external_debt': 0.2,
 'gdp_growth': 0.2,
 'inflation': 0.2,
 'reserves_months': 0.2}

In [5]:
# Ponderación de cada columna
df_norm['risk_score'] = sum(df_norm[col] * weight for col, weight in weights.items())
df_norm['risk_score'] = df_norm['risk_score'] * 100  # escalar a 0-100
df_norm = df_norm.reset_index()
df_norm[['country', 'risk_score']]


,country,risk_score
0,Brazil,39.038559
1,Colombia,70.420854
2,Mexico,59.358124
3,Peru,23.164129


In [6]:
df_norm[['country', 'risk_score']].to_csv('../data/processed/country_risk_score.csv', index=False)
print("✅ Índice de riesgo guardado en country_risk_score.csv")


✅ Índice de riesgo guardado en country_risk_score.csv
